<a href="https://colab.research.google.com/github/photochemistry/andersan-api/blob/main/%E5%8D%B3%E6%99%82%E5%9C%B0%E5%9B%B3%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colabでも動くように修正

In [ ]:
try:
    # On Google Colab
    from google.colab import drive

    drive.mount("/content/drive")
except:
    pass

In [ ]:
from andersan_backend import observes
from logging import getLogger, basicConfig, INFO, DEBUG

basicConfig(level=DEBUG)
# 特定の日時の各地理院メッシュ点での測定値をとってくる例
isodate = "2024-09-03T11:00+09:00"
zoom = 12
table = observes("kanagawa", isodate, zoom)
table

DEBUG:root:["kanagawa", "2024-09-03T11:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--


,lon,lat,X,Y,Z,NMHC,OX,NOX,TEMP,WX,WY
timestamp,,,,,,,,,,,
2024-09-03 11:00:00+09:00,138.867188,35.675147,3628,1613,12,5.009667,29.662173,4.168914,NaN,3.645199,6.866597
2024-09-03 11:00:00+09:00,138.955078,35.675147,3629,1613,12,6.061962,29.931887,4.034056,NaN,7.417923,8.920602
2024-09-03 11:00:00+09:00,139.042969,35.675147,3630,1613,12,7.114257,29.801940,4.254478,NaN,9.713605,7.276378
2024-09-03 11:00:00+09:00,139.130859,35.675147,3631,1613,12,13.069004,28.809011,6.124183,NaN,2.716571,-11.925141
2024-09-03 11:00:00+09:00,139.218750,35.675147,3632,1613,12,14.828828,30.816232,9.889436,225.103637,-0.923588,-3.181381
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-03 11:00:00+09:00,139.394531,35.245619,3634,1619,12,9.634303,29.094636,13.409776,NaN,10.525069,23.478489
2024-09-03 11:00:00+09:00,139.482422,35.245619,3635,1619,12,9.630303,28.063244,10.740799,248.469333,15.727899,23.622243
2024-09-03 11:00:00+09:00,139.570312,35.245619,3636,1619,12,9.626304,29.742651,5.197268,250.379126,21.481590,22.785760


辞書かJSONに整理したい。

In [ ]:
import json
import numpy as np

def dictize(df):
    spec = dict()
    loc = ("X", "Y", "lon", "lat", "Z")
    for col in loc:
        spec[col] = sorted(df[col].unique())
    spec["timestamp"] = sorted(df.index.unique())

    data = dict()
    data["XY"] = df[["X","Y"]].to_numpy().tolist()
    cols = df.columns
    for col in cols:
        if col not in loc:
            data[col] = list(df[col])
    return dict(spec=spec, data=data)

    #     res.setdefault(row[:5], {})[row.timestamp] = row.tide
    # j = json.dumps([dict(zip(groupers, k)) | {'Tide-Data': v} for k, v in res.items()])
    # return j

jsonify(table)

{'spec': {'X': [3628,
   3629,
   3630,
   3631,
   3632,
   3633,
   3634,
   3635,
   3636,
   3637,
   3638],
  'Y': [1613, 1614, 1615, 1616, 1617, 1618, 1619],
  'lon': [138.8671875,
   138.955078125,
   139.04296875,
   139.130859375,
   139.21875,
   139.306640625,
   139.39453125,
   139.482421875,
   139.5703125,
   139.658203125,
   139.74609375],
  'lat': [35.24561909420682,
   35.31736632923787,
   35.389049966911664,
   35.4606699514953,
   35.532226227703376,
   35.6037187406973,
   35.67514743608467],
  'Z': [12],
  'timestamp': [Timestamp('2024-09-03 11:00:00+0900', tz='UTC+09:00')]},
 'data': {'XY': [[3628, 1613],
   [3629, 1613],
   [3630, 1613],
   [3631, 1613],
   [3632, 1613],
   [3633, 1613],
   [3634, 1613],
   [3635, 1613],
   [3636, 1613],
   [3637, 1613],
   [3638, 1613],
   [3628, 1614],
   [3629, 1614],
   [3630, 1614],
   [3631, 1614],
   [3632, 1614],
   [3633, 1614],
   [3634, 1614],
   [3635, 1614],
   [3636, 1614],
   [3637, 1614],
   [3638, 1614],
   [3

指定時刻のデータをとってきて地図にする流れ。

In [ ]:
from folium import Map, LayerControl  # , Choropleth, FeatureGroup
from andersan_backend import prefecture_ranges
from andersan_map import map_layer
import numpy as np
from andersan import tile

# 日時の指定
isodate = "2024-08-31T00:00+09:00"
zoom = 12
table = observes("kanagawa", isodate, zoom)
# shape = len(table.Y.unique()), len(table.X.unique())
pref_range = prefecture_ranges["kanagawa"]
tiles, shape = tile.tiles(zoom, pref_range)
# mapに重ねるレイヤ
layers = []

# 測定値
for item in ["NMHC", "OX", "NOX", "TEMP", "WX", "WY"]:
    values = table[item].to_numpy()
    layers.append(map_layer("kanagawa", values.reshape(shape), item))

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# GroupedLayerControl({"NOX": layers[0]}, ["NOX"]).add_to(m)

# 表示
m

DEBUG:root:["kanagawa", "2024-08-31T00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--


6 項目を空間方向で補間し、欠測格子点がなくなった。これを毎日 24 時間前まで入手する。


次は、各格子点の予測値を OpenMeteo から入手して、同じように表示する。

G0L は OM から何を読んでいるか。(datatype1)

```
            "temperature_2m",
            "cloud_cover",
            "pressure_msl",
            "shortwave_radiation",
```

と`weathercode`

OpenMeteo の API では、指定された緯度経度とはすこしずれた場所の情報が送られてきているように思えるがそんなことはないか?


In [ ]:
import numpy as np
from logging import getLogger, basicConfig, INFO, DEBUG
from andersan_backend import openmeteo_tiles, prefecture_ranges, OPENMETEO_ITEMS
import datetime
from folium import Map, LayerControl  # , Choropleth, FeatureGroup
from andersan_map import map_layer
from andersan_backend import prefecture_ranges

basicConfig(level=INFO)

zoom = 12  # 14にしてもOpenMeteoのほうがそこまで解像度がない。
isodate = "2024-08-31T00:00+09:00"
all_forecast_dataframe = openmeteo_tiles("kanagawa", isodate, zoom)
# shape = len(all_forecast_dataframe.Y.unique()), len(all_forecast_dataframe.X.unique())
dt = datetime.datetime.fromisoformat(isodate)
pref_range = prefecture_ranges["kanagawa"]
tiles, shape = tile.tiles(zoom, pref_range)

# レイヤごとに作画する
layers = []
for item in OPENMETEO_ITEMS:
    values = all_forecast_dataframe[all_forecast_dataframe.date == dt][
        [item]
    ].to_numpy()[:, 0]

    # 新しいFoliumレイヤを定義する
    layers.append(map_layer("kanagawa", values.reshape(shape), item))

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# 表示
m

DEBUG:root:["kanagawa", "2024-08-31", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'openmeteo.sqlite'
DEBUG:sqlitedict:closing SqliteDict(openmeteo.sqlite)
DEBUG:sqlitedict.SqliteMultithread:received: --close--, send: --no more--
/AIR/andersan-api/andersan_map.py:9: RuntimeWarning: invalid value encountered in divide
  normed_data = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array))


本番テストに移行する前に: 標準化の方法を統一する。各測定量の標準化方法を固定し、データの多寡によらず同じ標準化が行われるようにする。


In [ ]:
import pandas as pd

# import matplotlib.pyplot as plt
# import folium
import numpy as np

# # import requests_cache
# # from retry_requests import retry
from logging import getLogger, basicConfig, INFO, DEBUG
from andersan_backend import (
    observes,
    openmeteo_tiles,
    prefecture_ranges,
    OPENMETEO_ITEMS,
    X_instant,
)
import datetime

# from folium import Map, LayerControl  # , Choropleth, FeatureGroup

basicConfig(level=DEBUG)
# 日時の指定
isodate = "2024-09-06T06:00+09:00"
zoom = 12

logger = getLogger()

X = X_instant("kanagawa", isodate, zoom)
X

DEBUG:root:["kanagawa", "2024-09-05T07:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:requests_cache.backends:Initializing backend: None airpollution
DEBUG:requests_cache.backends.base:Initialized SQLiteDict with serializer: SerializerPipeline(name=pickle, n_stages=2)
DEBUG:requests_cache.backends.sqlite:Opening connection to /AIR/andersan-api/airpollution.sqlite:responses
DEBUG:requests_cache.backends.base:Initialized SQLiteDict with serializer: None
DEBUG:requests_cache.backends.sqlite:Opening connection to /AIR/andersan-api/airpollution.sqlite:redirects
DEBUG:requests_cache.policy.actions:Cache directives from request headers: CacheDirectives()
DEBUG:requests_cache.policy.actions:Pre-read cache checks: Passed
DEBUG:requests_cache.policy.actions:Post-read cache actions: CacheActions(expire_after=-1, send_request=True)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.pref.kanagawa.jp:443
DEBUG:urllib3.connectionpool:http

{'Input_lookbacks': array([[[-1.55676165, -0.96701209, -0.54626565,         nan,
           0.03917042,  0.17160867],
         [-1.42468538, -0.71908419, -0.88073873,         nan,
           0.26269415, -0.03995094],
         [-1.40958104, -0.2196037 , -0.93154003,         nan,
           0.3926306 ,  0.25676668],
         ...,
         [-2.2061127 , -1.03667451, -0.58715193,         nan,
          -0.39072215,  0.31319549],
         [-0.97616302, -1.0115841 , -0.39684168,         nan,
          -0.15224642,  0.34063006],
         [-1.13670887, -1.18721695, -0.46102554,         nan,
          -0.20918533,  0.34618602]],
 
        [[-1.61913565, -1.07938549, -0.5071517 ,         nan,
          -0.02334424,  0.24290684],
         [-1.45522229, -0.70936322, -0.86539   ,         nan,
           0.34545685,  0.06430254],
         [-1.41663623, -0.03848075, -0.97413494,         nan,
           0.58096012,  0.35524583],
         ...,
         [-2.31672375, -1.14705823, -0.50935518,         na

In [ ]:
from datetime import timedelta

# 実測の未来値
forecasts_table = pd.DataFrame()
timeorigin = datetime.datetime.fromisoformat(isodate)
for delta in range(1, 9):
    dt = timeorigin + timedelta(hours=delta)
    table = observes("kanagawa", dt.isoformat(), zoom)
    forecasts_table = pd.concat([forecasts_table, table], axis=0)

forecasts_table
# 14 sec

DEBUG:root:["kanagawa", "2024-09-06T07:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:root:["kanagawa", "2024-09-06T08:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:root:["kanagawa", "2024-09-06T09:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:sqlitedict:closing SqliteDict(observes.sqlite)
DEBUG:root:["kanagawa", "2024-09-06T10:00:00+09:00", 12]
DEBUG:sqlitedict:opening Sqlite table 'unnamed' in 'observes.sqlite'
DEBUG:requests_cache.backends:Initializing backend: None airpollution
DEBUG:requests_cache.backends.base:Initialized SQLiteDict with serializer: SerializerPipeline(name=pickle, n_stages=2)
DEBUG:requests_cache.backends.sqlite:Opening connection to /AIR/andersan-api/airpollution.sqlite:responses
DEBUG:requests_cache.backends.base:Initia

,lon,lat,X,Y,Z,NMHC,OX,NOX,TEMP,WX,WY
timestamp,,,,,,,,,,,
2024-09-06 07:00:00+09:00,138.867188,35.675147,3628,1613,12,6.655007,5.680688,9.670621,NaN,-0.065297,3.126931
2024-09-06 07:00:00+09:00,138.955078,35.675147,3629,1613,12,7.196985,4.050794,10.221657,NaN,-1.159430,0.990978
2024-09-06 07:00:00+09:00,139.042969,35.675147,3630,1613,12,7.738962,2.895522,9.720450,NaN,-1.820353,-0.054500
2024-09-06 07:00:00+09:00,139.130859,35.675147,3631,1613,12,11.739151,4.661485,4.312839,NaN,-0.166934,4.826696
2024-09-06 07:00:00+09:00,139.218750,35.675147,3632,1613,12,11.435240,5.319645,7.101581,232.297727,-0.905149,4.033307
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-06 14:00:00+09:00,139.394531,35.245619,3634,1619,12,8.851281,61.416135,3.918039,NaN,-2.482819,-9.611578
2024-09-06 14:00:00+09:00,139.482422,35.245619,3635,1619,12,8.222713,60.332069,4.534810,312.000000,-9.629203,-5.922778
2024-09-06 14:00:00+09:00,139.570312,35.245619,3636,1619,12,7.594144,61.197268,5.050282,312.831317,-14.114159,-4.927017


In [ ]:
from andersan import tile

# そのうちOXのみ
# 地理院メッシュの間隔
pref_range = np.array(prefecture_ranges["kanagawa"])  # lon,lat
tiles, shape = tile.tiles(zoom, pref_range)

y = np.zeros([len(tiles), 8])
for j, (tileX, tileY) in enumerate(tiles):
    y[j, :] = forecasts_table[
        (forecasts_table.X == tileX) & (forecasts_table.Y == tileY)
    ].OX.to_numpy()

y
# 0.1 sec

array([[  5.68068838,  10.68110013,  20.33865083,  36.15163318,
         46.6899602 ,  51.57412323,  55.78221983,  55.16973716],
       [  4.05079383,   7.93918529,  17.84489559,  34.5803242 ,
         43.11734631,  49.10966077,  56.07735842,  52.81083925],
       [  2.89552204,   5.76138048,  15.77432928,  33.30253812,
         40.16754008,  47.13328508,  57.08063339,  52.22977679],
       [  4.6614853 ,   8.19866015,  17.18242118,  33.86416872,
         41.25399145,  49.05200289,  61.98445589,  61.02965608],
       [  5.31964527,  12.89735584,  20.65231028,  34.23096245,
         42.89007074,  53.77452862,  63.71401203,  60.80163147],
       [  8.17519142,  18.01262342,  29.18781484,  40.56109614,
         55.56109614,  66.42612335,  71.31639738,  73.48535564],
       [ 10.37168085,  17.08100359,  26.6027475 ,  43.08697565,
         60.48422815,  70.01194412,  75.27962984,  75.60185081],
       [ 10.1260965 ,  14.750731  ,  27.8352439 ,  45.59024458,
         63.08024321,  71.2235346

In [ ]:
import keras

basename = "andersan0_1.py"
model = keras.models.load_model(f"../andersan-train/{basename}.best.keras")
# model.summary()

pred = model.predict(X)
pred**0.5
# 6.7 sec

2024-09-09 13:50:37.537147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 13:50:37.556285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 13:50:37.562171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 13:50:37.576873: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
DEBUG:tensorflow:Falling back to TensorFlow client; w

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step


array([[       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        n

In [ ]:
import folium
from folium import Map, LayerControl, Choropleth, FeatureGroup
import matplotlib.pyplot as plt
from andersan_map import map_layer


def colorize(array, cmap="viridis"):
    # 値を色に変換
    normed_data = array / 120
    cm = plt.cm.get_cmap(cmap)
    return cm(normed_data)


layers = []
foresee = 3
pref_range = prefecture_ranges["kanagawa"]

maps = {"observes": y[:, foresee - 1], "predicts": (pred**0.5)[:, foresee - 1]}
for name, values in maps.items():
    # 新しいFoliumレイヤを定義する
    layers.append(
        map_layer("kanagawa", values.reshape(shape), f"{name} +{foresee} hrs")
    )

# 地図の中心位置
center = np.mean(pref_range, axis=0)[::-1]

# 地図を作る
m = Map(center, zoom_start=10)

# レイヤをオーバーレイする
for layer in layers:
    layer.add_to(m)

# 切り替えボタン(もっと見易くしたい)
LayerControl().add_to(m)

# 表示
m

だいぶ簡潔にできた。